This notebook contains the different machine learning models on which the training data of the <i>food-101</i> dataset will be trained on. Algorithms with different complexities will be used here. Its always a good practice to start with simplest model and later trying complex ones. But before we get our hands dirty with modeling, one more step lies between EDA and modeling which is Feature Engineering. In a usual scenario, feature engineering should get its separate notebook but because the dataset is already clean, images are already arranged in proper folders, all food items have 1000 images(except for one data object as seen in EDA), and data is well split into training and test set so, there is not much to do in feature engineering. Also, if we have to make some changes in the dataset it might be based on the model we choose. 

Starting the first model.
# Support Vector Machine 

Support vector machine is discriminative classifier formally defined by a separating hyperplane.

<i>SVM</i> is one of the simples models that we can you for classification. Images of different size could impact the learning of <i>SVM</i>. However, this is just an assumption. To see if this assumption holds we can train SVM on 2 datasets and evaluate the performance. To achieve this let's create a copy dataset where all the images are stored as square and of size <b>300x300</b>. 

### Feature engineering for SVM

Converting the rectangular images to square can be achieved through two ways. Either by shrinking the dimensions or cutting them out. Resizing the dimension will keep all the information but will move the image away from real world example. For example, let's see how the smallest image in the dataset will look like if we resize it to be square. 

<b>Original Image</b>

In [ ]:
from matplotlib import pyplot as plt
from PIL import Image

%matplotlib inline

image = Image.open('../../data/raw/food-101/images/macarons/3247436.jpg')
plt.imshow(image)
plt.show()

<b>Resized Image</b>

In [ ]:
import numpy as np

# Taking square root of the length * breath
sqrWidth = np.ceil(np.sqrt(image.size[0] * image.size[1])).astype(int) 
im_resize = image.resize((sqrWidth, sqrWidth))
plt.imshow(im_resize)
plt.show()

This looks quite bad but still holds the information about the food. Let's see what happens when we cut the extra dimensions to make the image square.

In [ ]:
# Create a new square white image with dimenion equal to smaller side of original image
# then paste the original image over the white image
def make_square(image, max_size=600, fill_color=(0, 0, 0, 0)):
    x, y = image.size
    size = min(max_size, x, y)
    new_im = Image.new('RGB', (size, size), fill_color)
    new_im.paste(image, (int((size - x) / 2), int((size - y) / 2)))
    return new_im

new_image = make_square(image)
plt.imshow(new_image)
plt.show()

This looks more like a real image, infact this image removes noise from the original image. However, we lose information while cropping the image. 

For the later method of cropping an image we can do a little variation and create a new kind of square image. Instead of using the smaller side of the image, we can use the longer one and fill the extra space with black or white color to generate a square image.

In [ ]:
# Create a new square white image with dimenion equal to smaller side of original image
# then paste the original image over the white image
def make_big_square(image, min_size=256, fill_color=(0, 0, 0)):
    x, y = image.size
    size = max(min_size, x, y)
    new_im = Image.new('RGB', (size, size), fill_color)
    new_im.paste(image, (int((size - x) / 2), int((size - y) / 2)))
    return new_im

new_image = make_big_square(image)
plt.imshow(new_image)
plt.show()

This keeps all the information and convert the image to a square but also adds a lot more information. We don't know yest, whether this helps with learning or not. We can create new dataset of images in this format as well to compare performance of algorithm on. 

For the resized image, shrining the longer dimension up to a certain length makes sense. If ratio of dimension is very high then the resizing can be far from realism. Let's see how many of the images in the dataset have ratio of more than 2:1. 

In [ ]:
from tqdm import tqdm
import os

path = '../../data/raw/food-101/images'

imageCount = 0
fileNameList = []

for r, d, f in tqdm(os.walk(path)):
    for file in f:
        fileName = os.path.join(r, file)
        image = Image.open(fileName)
        # dividing the longer side of image with the smaller one
        ratio = (max(image.size[0],image.size[1]) / min(image.size[0],image.size[1])) 
        if(ratio >= 2):
            fileNameList.append(fileName)
            imageCount += 1

print("Number of images with ratio more than 2:1 are-" + str(imageCount))

So, there are 47 images which have a ratio of more than 2:1, which is nothing compared to the total of 100999 images. Let's display 3 images from the list.

In [ ]:
from IPython.display import Image as Images, display
display(Images(filename=fileNameList[0]))
display(Images(filename=fileNameList[21]))
display(Images(filename=fileNameList[45]))

These 47 images contains a lot of false images as well. Let's take a look at false images. But because the images are very few in number we don't need to delete anything.

In [ ]:
display(Images(filename=fileNameList[1]))
display(Images(filename=fileNameList[17]))
display(Images(filename=fileNameList[28]))
display(Images(filename=fileNameList[29]))

Although, this is good that only 47 images have dimension ratio of more than 2:1 but we don't know how many images are rectangle. To do the performance check of how different algorithms behave with different image sizes and scaling, we need to have a good quantity of images with rectangle shape. Let's count the number of images which are rectangle. 

In [ ]:
path = '../../data/raw/food-101/images'

rectangleImageCount = 0

for r, d, f in tqdm(os.walk(path)):
    for file in f:
        fileName = os.path.join(r, file)
        image = Image.open(fileName)
        if(image.size[0] != image.size[1]):
            rectangleImageCount += 1

print("Number of rectangle images are: " + str(rectangleImageCount))

There are 38793 images which are rectangle in the dataset, which is 38.4%. This number is high enough to see the change in learning performance based on different reshaping techniques. Let's start with creating first datasets.

### ImageShrink
First dataset contains all square images achieved by shrinking the longer dimension to match the shorter one. We made a copy of data set and will now replace each rectangular image with a square in this dataset.

In [ ]:
path = '../../data/raw/food-101/imagesShrink'

for r, d, f in tqdm(os.walk(path)):
    for file in f:
        fileName = os.path.join(r, file)
        image = Image.open(fileName)
        # Finding the shorter dimension
        shorterDimension = min(image.size[0],image.size[1])
        im_resize = image.resize((shorterDimension, shorterDimension))
        # Replacing the original images with resized one.
        im_resize.save(fileName, 'JPEG' )
            

Let's see if that worked. Displaying a random rectangular image from both directories.

In [ ]:
display(Images(filename='../../data/raw/food-101/images/apple_pie/693210.jpg'))
display(Images(filename='../../data/raw/food-101/imagesShrink/apple_pie/693210.jpg'))

This looks good. Moving onto creating another dataset with longer length cropped to fit the square size.

### ImageCrop

In [ ]:
pathCrop = '../../data/raw/food-101/imagesCrop/'

for r, d, f in tqdm(os.walk(pathCrop)):
    for file in f:
        fileName = os.path.join(r, file)
        image = Image.open(fileName)
        # cropping the image using the make_square function used earlier
        new_image = make_square(image)
        # Replacing the original images with resized one.
        new_image.save(fileName, 'JPEG' )

Things to remember:
* images where we will lose the information if we crop to make it rectangular
* using an algorithm to find the important part of food image which can be used to check the performance as well specially for cropped images.
* Checking performance with or without data augmentation
* after making it square, check performance with or without making all images of same size
* use 5 different classifier atleast to know the performance